*Za rad sa MinIO object store-om koristeći Python API, poučiti [dokumentaciju](https://min.io/docs/minio/linux/developers/python/API.html).* 

### Zadatak 1: Kreiranje i provjera postojanja bucketa

Definišite funkciju koja provjerava postojanje bucketa pod nazivom 'my-data'. Ako bucket ne postoji, kreirajte ga. Ako već postoji, ispišite poruku 'Bucket 'my-data' već postoji.

In [2]:
from minio import Minio

ACCESS_KEY = "minioadmin"
SECRET_KEY = "minioadmin"
ENDPOINT = "localhost:9000"
BUCKET_NAME = "my-data"

client = Minio(ENDPOINT, access_key=ACCESS_KEY, secret_key=SECRET_KEY, secure=False)

In [ ]:
def create_bucket(naziv_bucketa):
    if not client.bucket_exists(naziv_bucketa):
        client.make_bucket(naziv_bucketa)
        print(f"Bucket '{naziv_bucketa}' successfully created.")
    else:
        print(f"Bucket '{naziv_bucketa}' already exists.")

create_bucket(BUCKET_NAME)

### Zadatak 2: Upload datoteka

Definišite funkciju koja uploaduje datoteku "data.txt" u bucket "my-data".

In [ ]:
def upload_files_fput(bucket_name, files):
    for file in files:
        client.fput_object(bucket_name, file, file)

upload_files_fput(BUCKET_NAME, ["data1.txt", "data2.txt"])

### Zadatak 3: Preuzimanje datoteka

Definišite funkciju koja preuzima datoteku "data.txt" iz bucketa "my-data" i sprema je lokalno.

In [ ]:
def download_file_fget(bucket_name, object_name, file_name):
    client.fget_object(bucket_name, object_name, file_name)
    print(f"Object '{object_name}' successfully downloaded as '{file_name}'.")

download_file_fget(BUCKET_NAME, "data1.txt", "downloaded_data1.txt")

### Zadatak 4: Brisanje datoteka

Definišite funkciju koja briše datoteku "data1.txt" iz bucketa "my'data".

In [ ]:
def delete_file(bucket_name, file_name):
    client.remove_object(bucket_name, file_name)
    print(f"Object '{file_name}' successfully deleted.")

delete_file(BUCKET_NAME, "data1.txt")

### Zadatak 5: Listanje datoteka u bucket

Definišite funkciju koja ispisuje popis svih datoteka u bucketu "my-data".

In [ ]:
def list_files(bucket_name):
    objects = client.list_objects(bucket_name)
    print(f"Objects in bucket '{bucket_name}':")
    for obj in objects:
        print(obj.object_name)

list_files(BUCKET_NAME)

### Zadatak 6: Upload i učitavanje proizvoljnog Python objekta

Upload objekta: Definišite funkciju koja kreira proizvoljan Python objekt (npr. rječnik, listu, vlastitu klasu) i uploaduje ga u Minio bucket koristeći `.pkl` format.

Učitavanje objekta: Definišite funkciju koja učitava sačuvani objekt iz Minio bucketa i ispisuje ga ili koristi u ostatku koda.

In [ ]:
import pickle
import io

def load_data_from_minio(bucket_name, file_name):
    try:
        response = client.get_object(bucket_name, file_name)
        return pickle.loads(response.data)
    except Exception as e:
        return {}

def save_data_to_minio(bucket_name, file_name, data):
    try:
        bytes_data = io.BytesIO(pickle.dumps(data))
        client.put_object(
            bucket_name,
            file_name,
            bytes_data,
            length=len(bytes_data.getvalue())
        )
    except Exception as e:
        print(f"Error saving {file_name} to MinIO: {e}")


def create_and_upload_object(bucket_name, object_name):
    sample_object = {
        "name": "NN",
        "age": 25,
        "products": ["loan", "visa card", "insurance"],
    }
    save_data_to_minio(bucket_name, object_name, sample_object)
    print(f"Object '{object_name}' successfully uploaded to bucket '{bucket_name}'.")


def load_and_print_object(bucket_name, object_name):
    loaded_object = load_data_from_minio(bucket_name, object_name)
    print(f"Loaded object from bucket '{bucket_name}': {loaded_object}")

create_and_upload_object(BUCKET_NAME, "sample_object.pkl")
load_and_print_object(BUCKET_NAME, "sample_object.pkl")

### Zadatak 7: Kontinuirano ažuriranje i čuvanje Python objekta

Definišite funkciju koja omogućuje kontinuirano ažuriranje i čuvanje Python objekta u Minio bucket. Funkcija treba:

1. Prije svake iteracije, pokušati učitati postojeći Python objekt iz Minio bucketa. Ako objekt ne postoji, treba kreirati prazan objekt (npr. prazan rječnik ili listu). 

2. U for petlji s unaprijed definisanim brojem iteracija: 
    * Ažurirati učitani objekt (npr. dodati novi element, izmijeniti postojeći). 
    * Sačuvati ažurirani objekt nazad u Minio bucket.

In [ ]:
from datetime import datetime

def load_and_save_object(bucket_name, object_name, num_iterations):
    try:
        for i in range(num_iterations):
            loaded_object = load_data_from_minio(bucket_name, object_name)
            if loaded_object is None:
                loaded_object = {}
            print(f"Object before update: {loaded_object}")

            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            loaded_object["update_time_" + str(i)] = timestamp

            save_data_to_minio(bucket_name, object_name, loaded_object)
            print(f"Object after update: {loaded_object}\n")

    except Exception as e:
        print(f"An error occured in main loop: {e}")

load_and_save_object(BUCKET_NAME, "my-dict", 5)

### Zadatak: Kreiranje MinIO utils modula

Kreirati Python modul `minio_utils.py` koji će sadržati pomoćne funkcije za rad sa MinIO-m koristeći minio paket.

Modul treba da sadrži:
- Funkciju za kreiranje i konfiguraciju MinIO klijenta
- Funkciju za kreiranje bucketa
- Funkciju za upload i download pkl fajlova
- Funkciju za upload i download JSON fajlova

Testirati modul sa primjerima korištenja svake funkcije.